<a href="https://colab.research.google.com/github/pollyjuice74/marketGAT/blob/master/marketGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import datetime as dt
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

!pip install torch-geometric
from torch_geometric.data import Data, Batch
from torch_geometric.data import TemporalData, HeteroData

import os
if not os.path.exists('marketGAT'):
  pass
  !git clone https://github.com/pollyjuice74/marketGAT.git
os.chdir('marketGAT')

from utils import *
from acc import *
from model import *

In [3]:
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=730)
n = 18
prev_context = 324
sample_size = 342 # 324 + 18
batch_size = 64
alpha = 0.15

symbols = ['SPY', 'TBLA', "MS", "PBM", "TCRX", "URG", "UROY", "UEC", "TBLA"]#"AMZN", "TSLA", "AAPL", "GOOGL", "META", "GM", "MS"]
graph = build_graph(symbols, startDate, endDate)

sample_graph, curr_ixs = sample(graph, symbols, sample_len=7*30, pred_len=7*5, live=False)
sample_graph

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Retrying due to error: only integer tensors of a single element can be converted to an index
Retrying due to error: only integer tensors of a single element can be converted to an index
Retrying due to error: only integer tensors of a single element can be converted to an index
Retrying due to error: only integer tensors of a single element can be converted to an index
Retrying due to error: only integer tensors of a single element can be converted to an index
Retrying due to error: only integer tensors of a single element can be converted to an index
Retrying due to error: only integer tensors of a single element can be converted to an index


HeteroData(
  buy_price=3.2899999618530273,
  SPY={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  TBLA={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  MS={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  PBM={
    node_ids=[239],
    x_samp=[204, 5],
    x_pred=[35, 5],
  },
  TCRX={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  URG={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  UROY={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  UEC={
    node_ids=[245],
    x_samp=[210, 5],
    x_pred=[35, 5],
  },
  (TBLA, next_in_sequence, TBLA)={
    edge_index_samp=[2, 210],
    edge_index_pred=[2, 35],
  },
  (SPY, same_time, TBLA)={
    edge_index_samp=[2, 210],
    edge_index_pred=[2, 34],
  },
  (MS, next_in_sequence, MS)={
    edge_index_samp=[2, 210],
    edge_index_pred=[2, 35],
  },
  (SPY, same_time, MS)={
    edge_index_samp=[2, 210],
    edge_index_pred=[2

In [20]:


y, time_dict = movement(sample_graph)
y, time_dict


SPY
tensor([1.0038, 1.0026, 1.0009, 0.9985, 0.9998, 0.9993, 1.0016, 1.0031, 1.0016,
        1.0016, 1.0016, 1.0034, 1.0044, 0.9961, 0.9936, 0.9928, 0.9917, 0.9919,
        0.9923, 0.9921, 0.9943, 0.9931, 0.9852, 0.9862, 0.9860, 0.9825, 0.9843,
        0.9847, 0.9810, 0.9801, 0.9778, 0.9769, 0.9782, 0.9776, 0.9853])
TBLA
tensor([1.0182, 1.0137, 1.0137, 1.0061, 1.0030, 1.0030, 1.2766, 1.1368, 1.0881,
        1.0805, 1.0821, 1.0638, 1.0684, 1.0441, 1.0410, 1.0334, 1.0243, 1.0410,
        1.0471, 1.0578, 1.0851, 1.0790, 1.0471, 1.0502, 1.0380, 1.0289, 1.0304,
        1.0350, 1.0426, 1.0380, 1.0304, 1.0304, 1.0274, 1.0274, 1.0361])
MS
tensor([1.0120, 1.0155, 1.0155, 1.0167, 1.0225, 1.0175, 1.0300, 1.0472, 1.0386,
        1.0381, 1.0280, 1.0276, 1.0259, 1.0272, 1.0244, 1.0338, 1.0270, 1.0221,
        1.0235, 1.0274, 1.0326, 1.0330, 1.0193, 1.0109, 1.0099, 1.0065, 1.0073,
        1.0050, 0.9870, 0.9936, 0.9921, 0.9896, 0.9910, 0.9890, 0.9903])
PBM
tensor([1.0151, 1.1043, 1.0794, 1.1037, 1.076

({'SPY': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0.]),
  'TBLA': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0.]),
  'MS': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0.]),
  'PBM': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.

In [21]:
import torch
from torch.nn import Module, TransformerEncoder, TransformerEncoderLayer, ModuleList, MultiheadAttention, Linear, Dropout
from torch_geometric.nn import GATv2Conv, to_hetero


class MarketTransformer(Module):
        def __init__(self, metadata, hidden_dims=4*16, num_classes=3, num_heads=8, num_layers=2):
                super().__init__()
                assert  hidden_dims % num_heads == 0, "Hidden dims and num_heads don't match"
                self.embedding = torch.nn.Linear(4, hidden_dims) # learned h,l,o,c representations
                self.pos_encoder = PositionalEncoder(hidden_dims) # retain temporal order info
                self.self_attn = MultiheadAttention(hidden_dims, num_heads) # each stock will attend to itself

                self.gat_convs = GATModule(hidden_dims, num_heads, num_layers)

                # for capturing relations between different stocks over time
                encoder_layers = TransformerEncoderLayer(d_model=hidden_dims, nhead=num_heads)
                self.transformer_encoder = TransformerEncoder(encoder_layers, num_layers=num_layers)

                # one-hot prediciton of stock's movement
                self.classifier = Linear(hidden_dims, num_classes)
                # time ix prediction of when the stock will reach pos. returns
                self.time_prediction = Linear(hidden_dims, 1)

                # turn homogeneous gnn into hetero gnn
                self.gat_convs = to_hetero(self.gat_convs, metadata, aggr="sum")

        def forward(self, x_dict, edge_index_dict):
                for key, x in x_dict.items():
                        x = self.embedding(x)
                        x = self.pos_encoder(x)
                        x_dict[key], _ = self.self_attn(x,x,x)

                for conv in self.gat_convs:
                        x_dict = conv(x_dict, edge_index_dict)

                x, mask = self._to_transformer_input(x_dict)
                x = self.transformer_encoder(x, mask)

                class_out_dict, time_out_dict = dict(), dict()
                for key in x_dict.keys():
                        class_out_dict[key] = self.classifier(x)
                        time_out_dict[key] = self.time_prediction(x)

                return out_dict

        def _to_transformer_input(self, x_dict):
                ### TODO
                all_x, mask = [], []
                return x, mask

        def compute_utility(class_out_dict, time_out_dict):
                utility_dict = dict()
                for key in class_out_dict.keys():
                        P_return = torch.softmax(class_out_dict[key])
                        utility_dict[key] = P_return / time_out_dict[key]
                return utility_dict

        def select_top_stocks():
                pass


class GATModule(Module):
        def __init__(self, hidden_dims, num_heads, num_layers):
                super().__init__()
                self.gat_convs = ModuleList() # utilize graph structure of the data
                for _ in range(num_layers):
                        self.gat_convs.append( GATv2Conv(hidden_dims, hidden_dims//num_heads, heads=num_heads, add_self_loops=False) )

        def forward(self, x_dict, edge_index_dict):
                for conv in self.gat_convs:
                        x_dict = conv(x_dict, edge_index_dict)
                return x_dict


class PositionalEncoder(Module):
        def __init__(self, d_model, max_len=5000):
                super().__init__()
                pe = torch.zeros(max_len, d_model)
                position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
                div_term = torch.exp( torch.arange(0, d_model, 2).float() * (-torch.log( torch.tensor(1e5) ) / d_model) )
                pe[:, 0::2] = torch.sin(position * div_term)
                pe[:, 1::2] = torch.cos(position * div_term)
                self.register_buffer( 'pe', pe.unsqueeze(0).transpose(0,1) )

                self.dropout = Dropout(0.1)

        def forward(self, x):
                x = x + pe[:x.size(0), :]
                return self.dropout(x)


# (s, p)->(s, p/m + f/v, k+u, d)->(s, k+u)->(s, u)->(s,1)->(1,)

In [34]:
# Suppose
import torch
import torch.nn.functional as F
import torch.nn as nn
import time
import random
import copy

# from model import *
# from utils import *


def sample(graph, symbols, sample_len, pred_len, live):
    sample_graph = HeteroData()

    while True:
        # try:
            # random SPY sample index
            spy_ix = torch.randint(0, graph['SPY'].x.size(0) - sample_len - pred_len, (1,)) if not live else int(graph['SPY'].x.size(0) - sample_len - pred_len)
            curr_ix = spy_ix + sample_len

            # print( graph['SPY'].t.shape)
            # SPY graph
            sample_graph['SPY'].x = torch.cat([graph['SPY'].x[spy_ix:curr_ix + pred_len],
                                              graph['SPY'].t[spy_ix:curr_ix + pred_len]], dim=1)
            sample_graph['SPY'].node_ids = graph['SPY'].node_ids[spy_ix:curr_ix + pred_len]

            # SPY nodesIDs
            first_nodeID = graph['SPY'].node_ids[spy_ix]
            curr_nodeID = graph['SPY'].node_ids[curr_ix]
            last_nodeID = graph['SPY'].node_ids[curr_ix + pred_len]

            # SPY edges
            spy_edges = graph['SPY', 'next_in_sequence', 'SPY'].edge_index[:, spy_ix:curr_ix + pred_len] ###

            for sym in symbols:
                if sym == 'SPY':
                  continue

                print(graph)
                # Check if `node_ids` exist for the current symbol
                if not hasattr(graph[sym], 'node_ids'):
                    raise AttributeError(f"Graph for symbol {sym} does not have 'node_ids'.")

                # stock nodes ixs
                f_sym_ix = torch.where(graph[sym].node_ids == first_nodeID.item())[0] # First
                c_sym_ix = torch.where(graph[sym].node_ids == curr_nodeID.item())[0] # Current
                l_sym_ix = torch.where(graph[sym].node_ids == last_nodeID.item())[0] # Last

                # stock graph
                sample_graph[sym].x = torch.cat([graph[sym].x[f_sym_ix:l_sym_ix],
                                                graph[sym].t[f_sym_ix:l_sym_ix]], dim=1)
                sample_graph[sym].node_ids = graph[sym].node_ids[f_sym_ix:l_sym_ix]

                # stock edges
                sym_edges = graph[sym, 'next_in_sequence', sym].edge_index[:, f_sym_ix:l_sym_ix] ###
                same_time_edges =  graph['SPY', 'same_time', sym].edge_index[:, f_sym_ix:l_sym_ix -1] ###

                dicts, edge_ixs = make_dicts([spy_edges, sym_edges]) # spy, sym
                same_time_edges = same_time_ix(same_time_edges, dicts) # convert same time ixs from graph ixs to sample graph ixs
                # print(same_time_edges)

                # set edges
                sample_graph[sym, 'next_in_sequence', sym].edge_index = edge_ixs[1]
                sample_graph['SPY', 'same_time', sym].edge_index = same_time_edges

                # normalize stock data and split into sample and pred
                sample_graph[sym].x_samp, sample_graph[sym].x_pred, buy_price = normalize(sample_graph[sym].x, curr_ix=c_sym_ix - f_sym_ix, pred_ix=l_sym_ix - f_sym_ix) # shifts ixs by first stock ix

                sample_graph.buy_price = buy_price

                # split edges into sample and pred
                sample_graph[sym, 'next_in_sequence', sym].edge_index_samp = sample_graph[sym, 'next_in_sequence', sym].edge_index[:, :c_sym_ix - f_sym_ix]
                sample_graph[sym, 'next_in_sequence', sym].edge_index_pred = sample_graph[sym, 'next_in_sequence', sym].edge_index[:, c_sym_ix - f_sym_ix:l_sym_ix - f_sym_ix+1]

                sample_graph['SPY', 'same_time', sym].edge_index_samp = sample_graph['SPY', 'same_time', sym].edge_index[:, :c_sym_ix - f_sym_ix]
                sample_graph['SPY', 'same_time', sym].edge_index_pred = sample_graph['SPY', 'same_time', sym].edge_index[:, c_sym_ix - f_sym_ix:l_sym_ix - f_sym_ix+1]
                # print(sample_graph[sym, 'next_in_sequence', sym].edge_index_samp)

            # set SPY edges, normalize SPY data and split into sample and pred
            sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index = edge_ixs[0]
            sample_graph['SPY'].x_samp, sample_graph['SPY'].x_pred, _ = normalize(sample_graph['SPY'].x, curr_ix=curr_ix - spy_ix, pred_ix=curr_ix+pred_len - spy_ix) # shifts ixs by first spy stock ix

            # split 'next_in_sequence' edges into sample and pred
            sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index_samp = sample_graph[sym, 'next_in_sequence', sym].edge_index[:, :curr_ix - spy_ix]
            sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index_pred = sample_graph[sym, 'next_in_sequence', sym].edge_index[:, curr_ix - spy_ix:curr_ix+pred_len - spy_ix + 1]
            # print(sample_graph['SPY'].x_pred)

            # REMOVE REDUNDANT sample_graph.x #
            del sample_graph['SPY'].x
            del sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index

            for sym in symbols:
              if sym == 'SPY':
                continue

              del sample_graph[sym].x
              del sample_graph[sym, 'next_in_sequence', sym].edge_index
              del sample_graph['SPY', 'same_time', sym].edge_index

            if live:
                return sample_graph, spy_ix
            else:
                curr_ixs = {sym: len(sample_graph[sym].x_samp) for sym in symbols}
                return sample_graph, curr_ixs

        # except Exception as e:
        #     print(f"Retrying due to error: {e}")
        #     continue


class Stock:
    def __init__(self, sym, live, graph):
        # Metadata
        self.price = None # setted with live_price
        self.trailing_data = [None, 0]#, None] # trailing_price, active, used_ix
        self.live = live
        self.sym = sym
        # Graph
        self.sample_len = 7*30 # 7h * 30d
        self.pred_len = 7*5 # 7h * 5d
        self.sample_graph = None
        self.ix = None
        self.pct = None

        self.initialize_graph(graph)
        ########################


    def update(self):
        """
        Updates price, gets either the live price or the price
        at the current step of the sample_graph data object.
        """
        if self.live:
          price =  self.live_price()[0][2]
        else:
          #print(self.sample_graph[self.sym].x_pred)
          price = self.sample_graph[self.sym].x_samp[-1, 2].item() * self.sample_graph.buy_price
          print("Updated:", price, self.sym)

        self.price = price


    def live_price(self, interval='1m'):
        """ Gets info and sets live price of stock """
        s = yf.Ticker(self.sym)
        df = s.history(period='1d', interval=interval).iloc[-1]

        self.price = float(df['Close'].item()) # CLOSE price set as LIVE PRICE

        x =  torch.tensor(df[['High', 'Low', 'Close', 'Open']])
        t = None #df.index.strftime('%H%M%S').astype('int64')
        node_ids = None #torch.tensor(df.index.strftime('%Y%m%d%H%M%S').astype('int64'), dtype=torch.int64)

        return (x, t, node_ids)  # x,t,node_ids,edge_index


    def sample_live(self, graph, sym): ###
        """ Sample live stock prices """
        self.update() # sets live price

        sample_graph, ix = sample(graph, sym, self.sample_len, self.pred_len, live=self.live)
        return sample_graph, ix


    def initialize_graph(self, graph):
        """ Initialize sample_graph, ix, pct and price """
        if self.live:
            self.sample_graph, self.ix = self.sample_live(graph, self.sym)
        else:
            self.sample_graph, self.ix = sample(graph, self.sym, self.sample_len, self.pred_len, self.live)
            self.update()

        pct = None
        self.pct = pct

        self.trailing_data[0] = self.sample_graph.buy_price # missing used_ix


class Account:
    def __init__(self, hidden_dims=7*30, epochs=2000, learning_rate=1e-05, live=False):
        # Account info
        self.history = list() # Tape of transaction history
        self.net_value = 416.29
        self.portions = None # [1, 8]
        self.used = None # [1, 8]
        self.bets = 2 # Limited ammount of bets per day
        # Data
        self.symbols = ['SPY', 'TBLA', "MS", "PBM", "TCRX", "URG", "UROY", "UEC", "TBLA"]#"AMZN", "TSLA", "AAPL", "GOOGL", "META", "GM", "MS"]
        self.graph = build_graph(self.symbols, startDate, endDate)
        self.sample_graph = None
        # Bets
        self.live = live
        self.current_bets = {sym: [] for sym in self.symbols if sym != 'SPY'} # sym: [(amount in $, shares), ..., ]
        self.stocks = {sym: Stock(sym, live=live, graph=self.graph) for sym in self.symbols} # Graphs of stocks
        # Model
        self.model = MarketTransformer(metadata=self.graph.metadata())
        # Training
        self.epochs = epochs
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)
        self.criterion = nn.BCEWithLogitsLoss()
        ##################################################


    def get_portion(self):
        """
        Portions out the Account.net_value as follows:

            Tithe:
                10% - Always cash
            Investment Capital:
                90% - Divided into 8 portions of 11.25% each (90/8)
        """

        if self.portions is None:
            inv_cap = self.net_value * 0.9 # Investment capital is 90%
            self.net_value -= inv_cap

            self.portions = torch.tensor([inv_cap / 8 for _ in range(8)], dtype=torch.float) # [1, 8]
            self.used = torch.zeros_like(self.portions)



        ixs = (self.used==0).nonzero(as_tuple=True)[0].tolist()# first portion that != 0 or negative,
        ix = random.choice(ixs)
        amount = torch.clone(self.portions[ix]) # select amount $

        self.portions[ix] -= amount.item() # to 0
        print("GET PORTION: ", self.portions, amount, ix)
        self.used[ix] = 1 # mark used vector

        return amount, ix


    def buy(self, sym, amount, ix):
        """
        Buys stock of a symbol given the amount of $ provided by the ix'th portion.

        Updates:
            self.stocks
            self.current_bets
            self.history
            self.portions
        """
        stock = self.stocks[sym]

        stock.update()
        price = stock.price

        if amount > price:
          shares = int(amount / price)
          left_over = (amount % price).item()
        else:
          print(sym, " stock price too high ", price, " for amount ", amount.item())
          self.portions[ix] += amount
          self.used[ix] = 0
          return

        # print(shares * stock.price, shares, ix)
        self.current_bets[sym].append((shares * price, shares, ix)) # sym: amount in stocks owned

        self.history.append((shares * price, shares, ix, 'Buy'))

        print("BUYING: ", sym, " for ", shares*price, " with ", left_over, " left over.")
        self.portions[ix] += left_over


    def sell(self, sym):
        """
        Sells all stock portions of a given symbol at the current stock price
        and update portions accordingly.

        Updates:
            self.used
            self.portions
            self.history
            self.current_bets
        """
        stock = self.stocks[sym]
        stock.update()

        for purchase_am, shares, ix in self.current_bets[sym]:
            self.used[ix] = 0 # set used vector space free
            self.portions[ix] += shares * stock.price

            self.history.append((shares * stock.price, shares, ix, 'Sell'))
            print("SOLD: ", sym, " for ", shares*stock.price, "*******************************************************************************************************************************************************************************************************************")

        self.current_bets[sym] = list() # remove bet


    def wait(self):
        """ Wait some time before continuing """
        time.sleep(10)


    def buy_conditions(self, pred):
        """ Test if preditction is for upward movement """
        return torch.equal(pred, torch.tensor([1,0,0]))


    def rank(self, ):
        """
        Ranks stock symbols by most to least profitable given a x day interval.

        The ranking occurs through simulated trading of the x day interval where
        the 'model' buys a stock if it
        """
        pass


    def print(self, i, epoch_loss, freq=25):
        """ Prints epoch and loss data """
        if i % freq == 0:
            print(f"Epoch: {i+1}/{self.epochs}, Loss: {epoch_loss}")


    def test(self): # Live trading simulation
        """
        Simulates the model predicting movement of the stocks given a sample interval
        and awaits for it to sell before beggining the process again.

        Goes in sequential order of a larger sample interval of a stock's history
        and gives a profitablility calculation based on model predictions.
        """
        while True:

          # Checks for buying conditions in all symbols
          for sym in [sym for sym, curr_bets in self.current_bets.items() if not curr_bets and sym != 'SPY']: #self.symbols:
              print("Checking for buying conditions: ", sym)

              stock = self.stocks[sym]
              stock.pct = torch.std(stock.sample_graph[sym].x_samp) * 0.5 # try to predict a half std dev movement

              # Train model
              ################################
              # latest = stock.live_price()[0][2] if self.live else sample_graph[sym].x_pred[0]

              y = torch.tensor([1, 0, 0]) #movement(latest, pct=stock.pct.item())
              pred = torch.clone(y) #self.model(sample_graph, pct=stock.pct)

              # loss = self.criterion(pred, y)

              # self.optimizer.zero_grad(set_to_none=True)
              # loss.backward()
              # self.optimizer.step()
              ################################

              if self.buy_conditions(pred):
                  if self.used != None and torch.all(self.used.bool()):
                    print("No more portions left.")
                    break

                  amount, used_ix = self.get_portion()
                  self.buy(sym, amount, used_ix) #if ix is None

          ### Inbetween period ######
          if self.live:
            self.wait()

          self.update_graph()
          ###########################

          # Checks for selling conditions in all symbols
          for sym in self.symbols:
              if sym == 'SPY':
                continue

              stock = self.stocks[sym]
              print("Monitoring: ", sym, "at ", stock.pct.item()*100, "%")

              # Sell at profit or sell at the end of the prediction time
              #print(stock.sample_graph[sym].x_pred, torch.any(stock.sample_graph[sym].x_pred))
              if self.pull_back(sym) or not torch.any(stock.sample_graph[sym].x_pred): # x_pred != empty
                  self.sell(sym) # sells all stocks owned of that symbol

                  if not torch.any(stock.sample_graph[sym].x_pred):
                    stock.sample_graph, stock.ix = sample(self.graph, sym, stock.sample_len, stock.pred_len, live=self.live)
                    self.stocks[sym] = stock # save new sample_graph


          self.print_acc()


    def print_acc(self):
        """ Print info for the account """

        if self.portions != None:
          net_val = self.net_value + torch.sum(self.portions) + sum([bet[0] for sym, bets in self.current_bets.items() for bet in bets])
        else:
          net_val = self.net_value

        for stock in self.stocks.values():
          if stock is not None:
              stock.update()

        print(
              f"""
                Account net_value: {net_val}\n

                Portions:
                {self.portions}\n
                Used:
                {self.used}\n

                Current bets: {self.current_bets}\n
                Trailing prices:\n
                { {sym: stock.trailing_data[0] for sym, stock in self.stocks.items() if stock != None and sym != 'SPY'} }

                LIVE PRICES:
                { {sym: stock.price for sym, stock in self.stocks.items() if stock is not None} }
                """
        )


    def update_graph(self): ###
        """
        Update graph to append live stock prices
        or step each stock's sample_graph
        """
        if self.live:
          for sym in self.graph.metadata()[0]:
            stock = self.stocks[sym]
            x, t, node_ids = stock.live_price()

            _, ix = self.graph[sym, 'next_in_sequence', sym].edge_index[:, -1]
            ix = ix.item()
            edge_index = torch.tensor([[ix],
                                       [ix+1],])

            # x, t, node_ids, edge_index
            self.graph[sym].x = torch.cat([self.graph[sym].x, x.unsqueeze(0)], dim=0)
            self.graph[sym].edge_index = torch.cat([self.graph[sym, 'next_in_sequence', sym].edge_index, edge_index], dim=1)
            # self.graph[sym].t = torch.cat([self.graph[sym].t, t])
            # self.graph[sym].node_ids = torch.cat([self.graph[sym].node_ids, node_ids])

        else:
          for sym in self.graph.metadata()[0]:
            stock = self.stocks[sym]

            if stock.ix < stock.sample_graph['SPY', 'same_time', sym].edge_index.shape[1]:
              stock.sample_graph, stock.ix = step(stock.sample_graph, stock.ix-1, sym)
              self.stocks[sym] = stock # update stocks

              stock.update()


    def pull_back(self, sym, pull=0.02):
        """
        Implements pull-back trading where you trail the price
        so it won't drop past a certain percent change since it's peak.

        target_price [float]: peak price achieved
        pull [float]: pull back percent from target_price
        """
        stock = self.stocks[sym]

        trailing_price, active = stock.trailing_data
        target_price = stock.sample_graph.buy_price * (1+stock.pct) # buy_price * 1.03

        if trailing_price is None:
            stock.trailing_data[0] = trailing_price = target_price.item()

        if stock.price >= target_price or active:
            #print(stock.trailing_data)
            stock.trailing_data[1] = 1 # set 'active' True

            if stock.price < trailing_price * (1-pull): # SELL CONDITION
                stock.trailing_data = [None, 0] # reset symbol's trailing price once sold
                return True

            if stock.price > trailing_price:
                #print(stock.trailing_data)
                stock.trailing_data[0] = stock.price # update 'trailing_price'

        return False


    def train(self):
        """
        Trains model to predict a one hot label of size [1, 3]
        corresponding to the stock's movement given a x day interval.
        """
        print("***")
        for i in range(self.epochs):
            epoch_loss = 0

            for _ in range(100): # sample 100 times
                print(self.symbols)
                sample_graph, curr_ixs = sample(self.graph, self.symbols, sample_len=self.sample_len, pred_len=self.pred_len, live=False) # Samples Training graph nodes
                curr_ixs = None

                while sample_graph['SPY'].x_pred.numel() > 0: # while there is more data to predict
                    y_hat_dict, time_dict_hat = self.model(sample_graph.x_samp_dict, sample_graph.edge_index_samp_dict) ###
                    y_dict, time_dict = movement(sample_graph[sym].x_pred[0], pct=pct.item())

                    # Calculate pct chg and time pred loss for each sym
                    for sym in sample_graph.node_types:
                        loss_y = self.criterion(y_hat_dict[sym], y_dict[sym])
                        loss_time = self.criterion(time_dict_hat[sym], time_dict[sym])
                        loss = loss_y + loss_time

                        epoch_loss += loss.item()

                        # Backward pass
                        self.optimizer.zero_grad(set_to_none=True)
                        loss.backward()
                        self.optimizer.step()

                    curr_ixs = step(sample_graph, curr_ixs, sym)

            self.print(i, epoch_loss, freq=25)


acc = Account()
acc.train()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


HeteroData(
  SPY={
    x=[3497, 4],
    t=[3498, 1],
    node_ids=[3498],
  },
  TBLA={
    x=[3497, 4],
    t=[3498, 1],
    node_ids=[3498],
  },
  MS={
    x=[3497, 4],
    t=[3498, 1],
    node_ids=[3498],
  },
  PBM={
    x=[924, 4],
    t=[925, 1],
    node_ids=[925],
  },
  TCRX={
    x=[3242, 4],
    t=[3243, 1],
    node_ids=[3243],
  },
  URG={
    x=[3497, 4],
    t=[3498, 1],
    node_ids=[3498],
  },
  UROY={
    x=[3497, 4],
    t=[3498, 1],
    node_ids=[3498],
  },
  UEC={
    x=[3497, 4],
    t=[3498, 1],
    node_ids=[3498],
  },
  (SPY, next_in_sequence, SPY)={ edge_index=[2, 3496] },
  (TBLA, next_in_sequence, TBLA)={ edge_index=[2, 3496] },
  (MS, next_in_sequence, MS)={ edge_index=[2, 3496] },
  (PBM, next_in_sequence, PBM)={ edge_index=[2, 923] },
  (TCRX, next_in_sequence, TCRX)={ edge_index=[2, 3241] },
  (URG, next_in_sequence, URG)={ edge_index=[2, 3496] },
  (UROY, next_in_sequence, UROY)={ edge_index=[2, 3496] },
  (UEC, next_in_sequence, UEC)={ edge_index

AttributeError: Graph for symbol S does not have 'node_ids'.

In [ ]:
#graph = build_graph(['AMZN', 'MSFT', 'TCRX'])
sample_graph, ix = sample(graph, sym)


#def sample_edges(sym):

# Symbol edges #
x_samp_len = sample_graph[sym].x_samp.shape[0]
ix = (sample_graph[sym, 'next_in_sequence', sym].edge_index == x_samp_len).nonzero(as_tuple=False)[0,1].item()

# set edges
edge_index_samp = sample_graph[sym, 'next_in_sequence', sym].edge_index[:, :ix-1]
edge_index_samp = torch.cat((edge_index_samp[:, 0].unsqueeze(1), edge_index_samp), dim=1) # mask dim

edge_index_pred = sample_graph[sym, 'next_in_sequence', sym].edge_index[:, ix-1:-1]


# SPY edges #
same_time = sample_graph['SPY', 'same_time', sym].edge_index
#print(same_time)

sp_same_ix, sym_ix = (same_time  == x_samp_len).nonzero(as_tuple=False)[:, 1].tolist()
sp_ix = (sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index == sp_same_ix).nonzero(as_tuple=False)[0,1].item()
print(sp_same_ix, sym_ix, sp_ix)

sp_ei_samp = sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index[:, :sp_ix] ###
sp_ei_pred = sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index[:, sp_ix:] ###
print(sp_ei_samp.shape, sp_ei_pred.shape)

sample_same_time = same_time[:, :sym_ix]
print(sample_same_time.shape, x_samp_len)



spy = sample_graph['SPY']
symb = sample_graph[sym]
print(sample_graph)


print([x.shape for k, x in symb.items()], [x.shape for k, x in spy.items()]) #edge_index_samp.shape, edge_index_pred.shape)
      #sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index)
#sample_graph['SPY', 'same_time', sym].edge_index = same_time_edges

# normalize stock data
ix, sample_graph[sym].x_samp.shape[0], sample_graph[sym].x_pred.shape, sample_graph['SPY'].x_samp.shape, sample_graph['SPY'].x_pred.shape

In [ ]:
sym = 'TCRX'





graph = build_graph(['AMZN', 'MSFT', 'TCRX'])
print(graph.node_types)
sample_graph, ix = sample(graph, sym)

sample_graph.x_samp_dict
sample_graph.edge_index_dict



# make edge index with sample graph ixs










# sample edges
# sample_graph[sym, 'next_in_sequence', sym].edge_index = graph[sym][sym, 'next_in_sequence', sym].edge_index[:, s_ix:curr_ix + pred_len] ###
# sample_graph['SPY', 'next_in_sequence', 'SPY'].edge_index = graph['SPY']['SPY', 'next_in_sequence', 'SPY'].edge_index[:, f_sp_ix:l_sp_ix] ###
# sample_graph['SPY', 'same_time', sym].edge_index =  graph['SPY', 'same_time', sym].edge_index[:, s_ix:curr_ix + pred_len] ###

# print(sample_graph['SPY'].x_samp.shape, ix)
# Check
# sp_ix_raw, s_ix_raw = sample_graph['SPY', 'same_time', sym][:, ix]
# graph[sym][sym].node_ids[s_ix_raw] == graph['SPY']['SPY'].node_ids[sp_ix_raw]

#sample_graph[sym].x_samp, sample_graph[sym].x_pred = step(sample_graph[sym].x_samp, sample_graph[sym].x_pred)
#print(sample_graph[sym].x_samp, sample_graph[sym].x_pred)
# print(sample_graph['SPY', 'same_time', sym][:, ix])
# print(sample_graph['SPY', 'next_in_sequence', 'SPY']['edge_index'][0])

# sp_y = movement(sp_x_pred, pct=0.003)
# stock_y = movement(stock_x_samp, pct=0.003)

# while len(sample_graph['SPY'].x_pred):
#   ix = step(sample_graph, ix, sym)
#   s_ix = len(sample_graph[sym].x_samp)
#   sp_ix = len(sample_graph['SPY'].x_samp)
#   print(s_ix, sp_ix)
#   print(sample_graph['SPY'].node_ids[sp_ix] == sample_graph[sym].node_ids[s_ix])
#   print(sample_graph['SPY'].node_ids[sp_ix], sample_graph[sym].node_ids[s_ix])

# print(sample_graph['SPY'].x_samp, sample_graph['SPY'].x_pred)
# print(sample_graph)